In [2]:
import pandas as pd
import csv
import os

import linecache
import sys

import matplotlib.pyplot as plt
import math
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

#from prettytable import PrettyTable

#簡轉繁用 
#https://pypi.python.org/pypi/hanziconv/0.2.1
from hanziconv import HanziConv

import matplotlib_venn 
from matplotlib_venn import venn2, venn2_circles


import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from numpy import mean, sqrt, square, arange

import pickle

import random
import datetime
import operator

#https://stackoverflow.com/questions/14519177/python-exception-handling-line-number
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))


In [23]:
# READ TAIWAN DATA WITH FEATURES
mysdata = pd.read_csv('./NameData/EthnicChineseMalaysian.csv', parse_dates = True)

In [24]:
mysdata.head()

,LastName,FirstName,Year,Age,Race,Name,IPAName,姓氏,Ethnic,FirstName1,...,FN2,FN1_sonin,FN1_muin,FN2_sonin,FN2_muin,FN1_sonin_index,FN2_sonin_index,FN1_muin_index,FN2_muin_index,YearRange
0,TEOH,Han Sen,1977.0,42.0,Chinese,TEOH HAN SEN,hɑn sɛn,張,Hokkien/Teochew,Han,...,10448,H,an,S,en,55,57,80,76,1
1,LIM,Yoke Foong,1977.0,42.0,Chinese,LIM YOKE FOONG,joʊk fuŋ,林,Hokkien,Yoke,...,10449,Y,oke,F,oong,56,60,230,250,1
2,ANG,Soo San,1977.0,42.0,Chinese,ANG SOO SAN,su sæn,洪,Teochew,Soo,...,10450,S,oo,S,an,57,57,231,80,1
3,CALVIN TAN,Kian Sin,1977.0,42.0,Chinese,CALVIN TAN KIAN SIN,ˈkiən sɪn,陳,Hokkien,Kian,...,10451,K,ian,S,in,58,57,21,75,1
4,CHAN,Khan Thai,1977.0,42.0,Chinese,CHAN KHAN THAI,kɑn taɪ,陳,Cantonese,Khan,...,10452,Kh,an,Th,ai,59,73,80,64,1


In [14]:
with open('./pkl_data/Totalname_list', 'rb') as handle:
    Totalname_list = pickle.loads(handle.read())
    
with open('./pkl_data/son_in_list', 'rb') as handle:
    son_in_list = pickle.loads(handle.read())

with open('./pkl_data/moe_data_dict', 'rb') as handle:
    moe_data_dict = pickle.loads(handle.read())
    
with open('./pkl_data/mu_in_list', 'rb') as handle:
    mu_in_list = pickle.loads(handle.read())

In [5]:
Path = './NameData/resampled_gender.csv'
resampled_gender = pd.read_csv(Path, dtype = 'str')

In [6]:
resampled_gender = resampled_gender.drop('userID',1)
resampled_gender = resampled_gender.drop('FN2_radical',1)
resampled_gender = resampled_gender.drop('FN1_radical',1)
resampled_gender = resampled_gender.drop('FN2_radical_index',1)
resampled_gender = resampled_gender.drop('FN1_radical_index',1)
resampled_gender = resampled_gender.drop('天格',1)
resampled_gender = resampled_gender.drop('人格',1)
resampled_gender = resampled_gender.drop('地格',1)
resampled_gender = resampled_gender.drop('外格',1)
resampled_gender = resampled_gender.drop('總格',1)

In [7]:
resampled_gender.head()

,FirstName,LastName,gender,message,name,FN1,FN2,mgender,FN1_sonin,FN1_muin,FN2_sonin,FN2_muin,FN1_sonin_index,FN2_sonin_index,FN1_muin_index,FN2_muin_index
0,民任,吳,1,1947,吳民任,335,297,1,m,ín,r,èn,11,26,113,97
1,昭呈,劉,1,1965,劉昭呈,462,111,1,zh,āo,ch,éng,8,21,84,56
2,恩田,莊,1,1959,莊恩田,86,202,1,NaN,ēn,t,ián,2,0,98,54
3,呈祥,林,1,1951,林呈祥,111,165,1,ch,éng,x,iáng,21,16,56,14
4,碧山,廖,1,1944,廖碧山,1287,447,1,b,ì,sh,ān,7,5,156,120


# Separate muin

In [8]:
mu_in_character = [['ā', 'ē','ī','ō','ū'], ['ǎ','ě','ǐ','ǒ','ǔ'], ['á','é','í','ó','ú'], ['à','è','ì','ò','ù']]
in_1 = ['ā', 'ē','ī','ō','ū']
in_2 = ['ǎ','ě','ǐ','ǒ','ǔ']
in_3 = ['á','é','í','ó','ú']
in_4 = ['à','è','ì','ò','ù']
in_5 = ['a','e','i','o','u']
in_s = ['ề', 'ǘ', 'ǚ', 'ǜ']
mu_dict = {}

In [9]:
mu_dict = {}
for mc in mu_in_character :
    for index,m in enumerate(mc):
        if index==0:
            mu_dict[m]='a'
        if index==1:
            mu_dict[m]='e'
        if index==2:
            mu_dict[m]='i'
        if index==3:
            mu_dict[m]='o'
        if index==4:
            mu_dict[m]='u'
mu_dict['ề'] = 'e'
mu_dict['ǘ'] = 'u'
mu_dict['ǚ'] = 'u'
mu_dict['ǜ'] = 'u'
            
# mu_dict

In [10]:
muin_dict = {}

for i in range(5):
    muin_dict[in_1[i]] = 1
    muin_dict[in_2[i]] = 2
    muin_dict[in_3[i]] = 3
    muin_dict[in_4[i]] = 4
    #muin_dict[in_5[i]] = 5
muin_dict['ề'] = 6
muin_dict['ǘ'] = 7
muin_dict['ǚ'] = 8
muin_dict['ǜ'] = 9
# muin_dict

In [11]:
resampled_gender['FN1_muin'] = resampled_gender['FN1_muin'].fillna(-1)
resampled_gender['FN2_muin'] = resampled_gender['FN2_muin'].fillna(-1)

In [ ]:
def seperate_pinjer(df,FN):
    mu_list = df[FN+'_muin'].tolist()
    new_mu_list = []
    for index,i in enumerate(mu_list):
        changed = False
        if i ==-1 or i=='-1':
            new_mu_list.append(i)
            continue
        for mu in muin_dict:
            if mu in i:
                #print(index,mu,i,muin_dict[mu])
                new_mu_list.append(i.replace(mu,mu_dict[mu]))
                changed = True
                break
        if changed == False :
            print(i)
            new_mu_list.append(i)
    df[FN+'_mu_in'] = new_mu_list
    return df

resampled_gender = seperate_pinjer(resampled_gender , 'FN1')
resampled_gender = seperate_pinjer(resampled_gender , 'FN2')


In [15]:
def add_pin_in_column(character, mode):
    #mode 1 = sonin
    #mode 2 = muin
    if character == -1:
        return -1
    term = Totalname_list[character]
    try:
        if term not in moe_data_dict:
            term = HanziConv.toTraditional(term)
        if term in moe_data_dict:
            for hete in moe_data_dict[term]['heteronyms']:
                if 'pinyin' in hete and  moe_data_dict[term]['title']!='啐':
                    word_p = hete['pinyin']
                    if '（'  in (hete['pinyin']):
                        if '（讀音）' in hete['pinyin']:
                            word_p = hete['pinyin'].replace('（讀音）','')
                        if '（語音）' in hete['pinyin']:
                            word_p = hete['pinyin'].replace('（語音）','')   
                        if '(' in word_p:
                            print(word_p + "!!")  
                    for mu in mu_in_list:
                        if mu in word_p:
                            if mode == 'sonin':
                                return word_p[:word_p.index(mu)]
                            else:
                                return mu                                    
            #找不到字音，看是否是哪個字的異體字
            for hete in moe_data_dict[term]['heteronyms']:
                for define in hete['definitions']:
                    if '異體字' in define['def']:
                        d =  define['def']
                        alt_term = d[  d.index('「')+1 : d.index('」') ]
                        #print(alt_term,term)
                        for hete2 in moe_data_dict[alt_term]['heteronyms']:
                            if 'pinyin' in hete2 and  moe_data_dict[alt_term]['title']!='啐':
                                word_p = hete2['pinyin']
                                if '（'  in (hete2['pinyin']):
                                    if '（讀音）' in hete2['pinyin']:
                                        word_p = hete2['pinyin'].replace('（讀音）','')
                                    if '（語音）' in hete2['pinyin']:
                                        word_p = hete2['pinyin'].replace('（語音）','')   
                                    if '(' in word_p:
                                        print(word_p + "!!")
                                for mu in mu_in_list:
                                    if mu in word_p:
                                        if mode == 'sonin':
                                            return word_p[:word_p.index(mu)]
                                        else:
                                            return mu     
            #print('在字典內但沒有拼音：',term)
        else:
            #print('不在字典內：',term)
            if term in special_word_dict:
                word_p = special_word_dict[term]['pinyin']
                for mu in mu_in_list:
                    if mu in word_p:
                        if mode == 'sonin':
                            return word_p[:word_p.index(mu)]
                        else:
                            return mu   
            else:
                #print('拼音不明：',term)
                if term not in unkown_dict:
                    unkown_dict[term] = 1 
                else:
                    unkown_dict[term] += 1

#             if len(moe_df[moe_df.字詞名.apply(lambda x: x==term)])>0:
#                 print('不在字典內：',term)
    except Exception as e:
        print (e)
        PrintException() 

def add_pin_in_index_column(pin_yin, mode):
    #mode 1 = sonin
    #mode 2 = muin
    if pin_yin == None:
        return -1
    try:
        if mode == 'muin':
            if pin_yin in mu_in_list:
                return mu_in_list.index(pin_yin)
            else:
                mu_in_list.append(pin_yin)
                return mu_in_list.index(pin_yin)

        if mode == 'sonin':
            if pin_yin in son_in_list:
                return son_in_list.index(pin_yin)
            else:
                son_in_list.append(pin_yin)
                return son_in_list.index(pin_yin)

    except Exception as e:
        print (e)
        print (fileName)
        PrintException()

In [17]:
# Only use 'mu' without then 'in'
resampled_gender['FN1_muin'] = resampled_gender['FN1_mu_in']
resampled_gender['FN2_muin'] = resampled_gender['FN2_mu_in']
resampled_gender = resampled_gender.drop('FN1_mu_in',1)
resampled_gender = resampled_gender.drop('FN2_mu_in',1)

resampled_gender['FN1_muin_index'] = resampled_gender.FN1_muin.apply(lambda x: add_pin_in_index_column(x,'muin'))
resampled_gender['FN2_muin_index'] = resampled_gender.FN2_muin.apply(lambda x: add_pin_in_index_column(x,'muin'))

In [18]:
resampled_gender.head()

,FirstName,LastName,gender,message,name,FN1,FN2,mgender,FN1_sonin,FN1_muin,FN2_sonin,FN2_muin,FN1_sonin_index,FN2_sonin_index,FN1_muin_index,FN2_muin_index
0,民任,吳,1,1947,吳民任,335,297,1,m,in,r,en,11,26,75,76
1,昭呈,劉,1,1965,劉昭呈,462,111,1,zh,ao,ch,eng,8,21,74,25
2,恩田,莊,1,1959,莊恩田,86,202,1,NaN,en,t,ian,2,0,76,21
3,呈祥,林,1,1951,林呈祥,111,165,1,ch,eng,x,iang,21,16,25,2
4,碧山,廖,1,1944,廖碧山,1287,447,1,b,i,sh,an,7,5,164,80


# RFC Gender

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def dataset_statistics(dataset):
    """
    Basic statistics of the dataset
    :param dataset: Pandas dataframe
    :return: None, print the basic statistics of the dataset
    """
    print (dataset.describe())
    
#http://dataaspirant.com/2017/06/26/random-forest-classifier-python-scikit-learn/
#Split train and test
def split_dataset(dataset, train_percentage, feature_headers, target_header):
    """
    Split the dataset with train_percentage
    :param dataset:
    :param train_percentage:
    :param feature_headers:
    :param target_header:
    :return: train_x, test_x, train_y, test_y
    """

    # Split dataset into train and test dataset
    train_x, test_x, train_y, test_y = train_test_split(dataset[feature_headers], dataset[target_header],
                                                        train_size = train_percentage)
    return train_x, test_x, train_y, test_y

def random_forest_classifier(features, target):
    """
    To train the random forest classifier with features and target data
    :param features:
    :param target:
    :return: trained random forest classifier
    """
    clf = RandomForestClassifier(n_estimators = 33, n_jobs = -1 , min_samples_leaf = 1)
    clf.fit(features, target)
    return clf

In [20]:
def basic_RFClassifier(data_df,x_feature,y_feature,Validation_times):

    #N fold cross-validation
    Train_accuracy_score = 0
    Test_accuracy_score = 0
    
    for i in range(Validation_times):
        train_x, test_x, train_y, test_y = split_dataset(data_df, 0.7, x_feature, y_feature)
 
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y)
        
        predictions = trained_model.predict(test_x)
#         print(predictions)

        # Train and Test Accuracy
        if(Validation_times!=1):
            print('Run times:', i+1)
        print ("Train Accuracy :: ", accuracy_score(train_y, trained_model.predict(train_x)))
        print ("Test Accuracy  :: ", accuracy_score(test_y, predictions))
        #print ("Confusion matrix\n ", confusion_matrix(test_y, predictions))

        Train_accuracy_score += accuracy_score(train_y, trained_model.predict(train_x))
        Test_accuracy_score += accuracy_score(test_y, predictions)
        #calculate_error(predictions, test_y )
        #validate_error(predictions, test_y, test_x , data_df)
        
    if(Validation_times != 1):
        Train_accuracy_score /= Validation_times
        Test_accuracy_score /= Validation_times
        print ("Average Train Accuracy :: ", Train_accuracy_score)
        print ("Average Test Accuracy  :: ", Test_accuracy_score)
    
    return trained_model

# Use 'gender'

In [21]:
# 6F

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['gender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.9620487576183778
Test Accuracy  ::  0.8573264781491002


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.9613924050632912
Test Accuracy  ::  0.8579554777662309


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.962037037037037
Test Accuracy  ::  0.8540994366351256


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.9624472573839662
Test Accuracy  ::  0.8561505223431604


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.9627051101734646
Test Accuracy  ::  0.8568068697697314
Average Train Accuracy ::  0.9621331458040319
Average Test Accuracy  ::  0.8561614614669365


In [25]:
# Read uni-gram classifier
# 6F

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
    
result = gender_test.predict(mysdata[x_feature])
prob = gender_test.predict_proba(mysdata[x_feature])

In [ ]:
gender_list = mysdata['GuessedGender'].tolist()

In [ ]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))